In [236]:
import numpy as np
import pandas as pd
from imutils import paths
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection
import random

In [424]:
all_df = pd.read_csv('.\\ipl_csv2\\all_matches.csv', index_col='match_id')
print(len(all_df['striker'].unique()))
print(len(all_df['bowler'].unique()))

540
422


In [237]:
dataPaths = list(paths.list_files('.\ipl_csv2'))

dataPaths.pop(len(dataPaths) - 2)
dataPaths.pop(len(dataPaths) - 1)

def match_history(p_team, opp_team, num):
    req_df = []
    
    for match_file in dataPaths:
        try:
            df = pd.read_csv(match_file, index_col='match_id')
            df = preprocess(df)

            inn1 = df[df['innings'] == 1]
            inn2 = df[df['innings'] == 2]

            if (inn1.iloc[1]['batting_team'] == p_team or p_team == -1):
                if (inn1.iloc[1]['bowling_team'] == opp_team or opp_team == -1):
                    if len(req_df) < num or num == -1:
                        req_df.append(df)
            elif (inn2.iloc[1]['batting_team'] == p_team or p_team == -1):
                if (inn2.iloc[1]['bowling_team'] == opp_team or opp_team == -1):
                    if len(req_df) < num or num == -1:
                        req_df.append(df)

        except Exception as e:
            print(e)

    return req_df


def preprocess(df):
    pp_df = df[df['ball'] <= 6.1]
    pp_df = pp_df[pp_df['innings'] <= 2]

    pp_df['total_runs'] = pp_df['runs_off_bat'] + pp_df['extras']

    return pp_df

In [451]:
dataPaths

2\\1136611.csv',
 '.\\ipl_csv2\\1136612.csv',
 '.\\ipl_csv2\\1136613.csv',
 '.\\ipl_csv2\\1136614.csv',
 '.\\ipl_csv2\\1136615.csv',
 '.\\ipl_csv2\\1136616.csv',
 '.\\ipl_csv2\\1136617.csv',
 '.\\ipl_csv2\\1136618.csv',
 '.\\ipl_csv2\\1136619.csv',
 '.\\ipl_csv2\\1136620.csv',
 '.\\ipl_csv2\\1175356.csv',
 '.\\ipl_csv2\\1175357.csv',
 '.\\ipl_csv2\\1175358.csv',
 '.\\ipl_csv2\\1175359.csv',
 '.\\ipl_csv2\\1175360.csv',
 '.\\ipl_csv2\\1175361.csv',
 '.\\ipl_csv2\\1175362.csv',
 '.\\ipl_csv2\\1175363.csv',
 '.\\ipl_csv2\\1175364.csv',
 '.\\ipl_csv2\\1175365.csv',
 '.\\ipl_csv2\\1175366.csv',
 '.\\ipl_csv2\\1175367.csv',
 '.\\ipl_csv2\\1175368.csv',
 '.\\ipl_csv2\\1175369.csv',
 '.\\ipl_csv2\\1175370.csv',
 '.\\ipl_csv2\\1175371.csv',
 '.\\ipl_csv2\\1175372.csv',
 '.\\ipl_csv2\\1178393.csv',
 '.\\ipl_csv2\\1178394.csv',
 '.\\ipl_csv2\\1178395.csv',
 '.\\ipl_csv2\\1178396.csv',
 '.\\ipl_csv2\\1178397.csv',
 '.\\ipl_csv2\\1178398.csv',
 '.\\ipl_csv2\\1178399.csv',
 '.\\ipl_csv2\\1178400.csv

In [410]:
p_team = "Kolkata Knight Riders"
opp_team = "Kings XI Punjab"
df_list = match_history(p_team, -1, -1)

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds


In [411]:
import csv

def read_files(fileName, n_features):
    dictionary = {}
    with open(fileName, mode='r') as infile:
        reader = csv.reader(infile)

        for row in reader:
            for i in range(1, n_features + 1):
                dictionary.setdefault(row[0], []).append(row[i])
    
    return dictionary
            

In [412]:
stad_rating = read_files('stadium_rating.csv', 1)

for k in stad_rating:
    stad_rating[k] = float(stad_rating[k][0])

print(stad_rating)

{'Rajiv Gandhi International Stadium, Uppal': 45.1875, 'Maharashtra Cricket Association Stadium': 47.285714285714285, 'Saurashtra Cricket Association Stadium': 54.9, 'Holkar Cricket Stadium': 51.611111111111114, 'M.Chinnaswamy Stadium': 45.72435897435897, 'Wankhede Stadium': 45.29054054054054, 'Eden Gardens': 46.103896103896105, 'M Chinnaswamy Stadium': 45.72435897435897, 'Feroz Shah Kotla': 47.57823129251701, 'Punjab Cricket Association IS Bindra Stadium, Mohali': 48.098214285714285, 'Green Park': 54.375, 'Punjab Cricket Association IS Bindra Stadium': 48.098214285714285, 'Rajiv Gandhi International Stadium': 45.1875, 'MA Chidambaram Stadium': 46.21186440677966, 'Sawai Mansingh Stadium': 44.88297872340426, 'Arun Jaitley Stadium': 47.57823129251701, 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium': 40.73076923076923, 'Sheikh Zayed Stadium': 43.310344827586206, 'Dubai International Cricket Stadium': 44.45454545454545, 'Sharjah Cricket Stadium': 46.916666666666664, 'MA Chidambaram S

In [413]:
batsmen_record = read_files('batsmen_record.csv', 2)

for k in batsmen_record:
    for (i, p) in enumerate(batsmen_record[k]):
        batsmen_record[k][i] = [int(s) for s in p[1:-1].split(',')]

print(len(batsmen_record))

292


In [414]:
bowlers_record = read_files('bowlers_record.csv', 4)

for k in bowlers_record:
    for (i, p) in enumerate(bowlers_record[k]):
        bowlers_record[k][i] = [int(s) for s in p[1:-1].split(',')]

print(len(bowlers_record))

346


In [445]:
def get_venue_score(venue):
    return stad_rating[venue]

def get_batsmen_scores(u_bat):
    rating = []

    for batsman in u_bat:
        rating.append(np.mean(batsmen_record[batsman][0]))
    
    return rating
    
def get_bowlers_scores(u_bowlers):

    rating = []

    for bowler in u_bowlers:
        try:
            rating.append(np.mean(bowlers_record[bowler][0]))
        except Exception as e:
            rating.append(30)
    
    return rating

def get_wickets(u_bat):
    return len(u_bat) - 2

# def get_team_rating(df):
#     date = df.iloc[0]['season']
#     if '/' in date:
#         date = date.split('/')[0]
    



In [446]:
X = pd.DataFrame()
Y = pd.DataFrame()

for df in df_list:
    match_id = df.index[0]

    #innings for which we are going to use for training (when the team is batting)
    inn = df.loc[(df['batting_team'] == p_team)]['innings'].iloc[0] 
    df_train_inn = df.loc[(df['innings'] == inn)]

    runs_x = df_train_inn['total_runs'].sum() #runs of training innings
    runs_target = df.loc[(df['innings'] == 3 - inn)]['total_runs'].sum() #runs of other innings



    #convert the df into usable training data with required features
    # to add team rating: 'team_rating':[get_team_rating(df_train_inn)], 

    venue = df_train_inn.iloc[1]['venue']
    u_bat = list(set(df_train_inn['striker'].unique()) | set(df_train_inn['non_striker'].unique()))
    u_bowlers = list(df_train_inn['bowler'].unique())

    temp = pd.DataFrame({'batsmen_rating': [np.sum(np.array(get_batsmen_scores(u_bat)))], 'bowler_rating': [np.sum(np.array(get_bowlers_scores(u_bowlers)))], 'wickets': [get_wickets(u_bat)]})
    
    X = X.append(temp)
    Y = Y.append(pd.DataFrame({'runs': [runs_x]}))

In [447]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, random_state=42)
train_x.head()

,batsmen_rating,bowler_rating,wickets
0,55.238944,53.408983,3
0,37.274512,50.714345,1
0,42.078755,47.970975,1
0,53.283197,55.102711,1
0,31.304054,57.572751,0


In [418]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge, Ridge
from sklearn.tree import DecisionTreeRegressor 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV, RepeatedKFold


In [448]:
linear = LinearRegression()
bayesian = BayesianRidge()
dtree = DecisionTreeRegressor(random_state = 0) 
rforest = RandomForestRegressor(n_estimators = 100, random_state = 0)
lasso = Lasso(alpha = 0.0083, random_state=1)
ENet = ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3)
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

models = [linear,bayesian,dtree,rforest, lasso, ENet, KRR, GBoost, model_lgb]

In [449]:
def get_predictions(model):
    model.fit(train_x, train_y)
    return model.predict(test_x)

In [450]:
for model in models:
    y_pred = get_predictions(model)
    print(mean_squared_error(y_pred, test_y, squared=False))

8.305871110477012
8.284391569387521
12.347531674854633
C:\Users\rishi\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<ipython-input-449-8482759cc9c3>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(train_x, train_y)
8.736604691667212
8.30434613413072
8.305704588350453
8.798637541768942
C:\Users\rishi\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
8.83794095076912
[LightGBM] [Warning] feature_fraction is set=0.2319, colsamp

In [210]:
#polynomial regression
polynomial_features= PolynomialFeatures(degree=2)
poly_reg = LinearRegression()
poly_reg.fit(polynomial_features.fit_transform(train_x), train_y)
y_pred_poly = poly_reg.predict(polynomial_features.fit_transform(test_x))
print(mean_squared_error(y_pred_poly, test_y, squared=False))

11.831544359919675
